# Database Migration with Alembic

In [ ]:
# Install dependencies
!pip install alembic
!pip install sqlmodel

In [ ]:
# alembic initialization
!alembic init alembic

Running `alembic init alembic` creates a 
- a folder alembic (migration environment)
    - versions: folder with all the migrations file
    - env.py: environement file (create the connections, get the table metadata)
    - script.py.mako: template of the migration file
- alembic.ini config file

In [ ]:
# Create the sqlite database 
import sqlite3
c = sqlite3.connect("project.db")

## Define the tables with SQLModel

In [2]:
%%writefile models.py

from typing import Optional
from sqlmodel import (
    Field,
    SQLModel,
    create_engine,
    Relationship
)

class User(SQLModel, table=True):
    __tablename__ = "users"
    __table_args__ = {'extend_existing': True}

    id: Optional[int] = Field(
        default=None,
        primary_key=True,
        index=True,
        nullable=False)
    first_name: str
    last_name: str
    number_of_cars: int
    team_id: Optional[int] = Field(
        index=True,
        foreign_key="teams.id",
    )

    team: "Team" = Relationship(back_populates="users",  sa_relationship_kwargs={"lazy": "selectin"},)


class Team(SQLModel, table=True):
    __tablename__ = "teams"
    __table_args__ = {'extend_existing': True}

    id: Optional[int] = Field(
        default=None,
        primary_key=True,
        index=True,
        nullable=False)
    name: str

    users: User = Relationship(back_populates="team",  sa_relationship_kwargs={"lazy": "selectin"},)


Overwriting models.py


## Alembic setup with sqlmodel

- Click folder left panel
- Open alembic.ini  
    - replace the line `sqlalchemy.url = sqlite:////content/project.db`

In [ ]:
!alembic current

In [ ]:
!alembic history

- Insert the following line `import sqlmodel` in the file script.py.mako in the import section
    - sqlmodel library is now automatically imported in every migration script.
- add the following lines in alembic/env.py
```
from sqlmodel import SQLModel
import models
target_metadata = SQLModel.metadata
```

This tells Alembic to watch the changes inside `models.py` that are stored in the SQLModel.metadata object


## List of alembic useful commands

In [ ]:
!alembic current

In [ ]:
!alembic history

In [ ]:

# creates a new file migration script in alembic/versions

# !alembic revision -m "create users and teams table" --autogenerate
# !alembic revision -m "remove number_of_cars in users table" --autogenerate
# !alembic revision -m "add email in user table" --autogenerate


In [ ]:
# update the state of the database to the latest revision (head)
!alembic upgrade head

In [ ]:
# downgrade to previous version

# !alembic downgrade -1

## Quick checks in the database using sql queries

In [ ]:
import pandas as pd
pd.read_sql_query("SELECT * from users", c)


In [ ]:
pd.read_sql_query("SELECT * from teams", c)


In [ ]:
pd.read_sql_query("SELECT * from alembic_version", c)

## Recommended protocol to follow for each migration
- Modifiy the models.py file to make the changes
- alembic history / current to check that the database is at head (latest)
- alembic revision 
- double check script 
- alembic upgrade 
- alembic history / current to check that the new revisio was successful 
- visualize changes in the database
- reload notebook to import the new models
- added data if needed with the updated models

## Exercises

In [ ]:
from sqlmodel import create_engine, Session
from models import User, Team
# url = "sqlite:///project.db"
url = "sqlite:////content/project.db"
engine = create_engine(url, echo=True)

Let's start a new project we need users

0. Create the table users and teams with your first migration script
    

1. Insert 2 users and 2 teams such that
    - user_a belongs to team_a
    - user_b belongs to team_b

In [ ]:
# TODO

We realized the number_of_cars field in the users table in not useful. Can we remove it

2. Create second migration script to remove the number_of_cars field
    - after upgrading to the latest migration version, try `alembic downgrade -1`. Check table again. Comment.
    - upgrade back to the latest migration using `alembic upgrade head`

It would be nice to add the email of each user in the users table. It's just {first_name}.{last_name}@axa.com

3. Create a third migration script to add the email column. 
    - Also we can automatically fill the new column using the existing ones. Let's try with a raw sql command inside the migration script using the method `op.execute(#insert raw sql command)` in the upgrade part of the migration script.
    - downgrading and upgrading back should give the same initial result (as the data migration is also done in this case)

## The End

-----

In [ ]:
# some bandaid commands

# c.execute("alter table users drop column 'email' ;")
# c.commit() 

# c.execute("update alembic_version set version_num = '9673525dbab5';")
# c.commit() 
